In [1]:
""" Script to take planning applications and match to OSAddressBase addresses

Written by: Christine Langston, April 2024
"""
import pandas as pd
import time
import numpy as np
import copy
import re

In [2]:
#create a function to iterate through csv. use for OSAddressBase file
def read_csv(file_name, columns):
    for chunk in pd.read_csv(file_name, chunksize=10000, usecols=columns, 
                            dtype={'urpn':str, 'parent_urpn': str, 'class': str, 'latitude': float, 
                                                  'longitude': float, 'country': str}):
    
        #if chunk['country'] == 'E':
        yield chunk


In [3]:
#takes a merged dataset and separates matched addresses and not matched
def separate_matches(dataset, column_name, match_strategy):
    dataset_match = dataset.drop(dataset[pd.isna(dataset[column_name]) == True].index)
    dataset_no_match = dataset.drop(dataset[pd.isna(dataset[column_name]) == False].index)
    dataset_match['match_strategy'] = match_strategy
    return dataset_match, dataset_no_match

In [4]:
#function to merge and then split based on if an address was matched  
def my_merge(left, right, left_on, right_on): 
    merged = left.merge(right, how = 'left', left_on = left_on, right_on = right_on)
    merged_match = merged.drop(merged[pd.isna(merged['uprn']) == True].index) 
    merged_no_match = merged.drop(merged[pd.isna(merged['uprn']) == False].index) 
    return merged_match, merged_no_match

In [5]:
%%time
#### READ IN THE DATA IF EXPORTED 
resi_AB = pd.read_csv('data/resi_AB_cleaned_parsed_110524.csv', low_memory = False)


CPU times: user 4min 50s, sys: 11min 20s, total: 16min 11s
Wall time: 33min 53s


In [6]:
%%time
other_AB = pd.read_csv('data/other_AB_cleaned_parsed_110524.csv',low_memory = False)



CPU times: user 41.6 s, sys: 10.6 s, total: 52.2 s
Wall time: 1min 17s


In [7]:
#load in the data that hasn't been matched yet
london_data = pd.read_csv('data/London2_no_match_2603.csv',low_memory = False)

Some data cleaning steps to clean up the Nan and numerical data coming from the CSV

In [8]:
### ------ DATA CLEANING  ----------
#edit the parent uprn from OSAddress base to fill out to be strings with 12 digits and leading zeros
#replace nan
resi_AB = resi_AB.replace([np.nan, -np.inf], 0)

#cast as integer
resi_AB['parent_uprn'] = resi_AB['parent_uprn'].astype('Int64')
resi_AB['uprn'] = resi_AB['uprn'].astype('Int64')


In [9]:
### ------ DATA CLEANING  ----------
#cast as string 
resi_AB['parent_uprn'] = resi_AB['parent_uprn'].astype(str)
resi_AB['uprn'] = resi_AB['uprn'].astype(str)

#fill in with left side padding zeros 
resi_AB['parent_uprn'] = resi_AB['parent_uprn'].apply(lambda x: '{0:0>12}'.format(x))
resi_AB['uprn'] = resi_AB['uprn'].apply(lambda x: '{0:0>12}'.format(x))


In [11]:
#do the same cleaning as above but for the OTHER addresses
other_AB = other_AB.replace([np.nan, -np.inf], 0)

#cast as integer
other_AB['parent_uprn'] = other_AB['parent_uprn'].astype('Int64')
other_AB['uprn'] = other_AB['uprn'].astype('Int64')

In [12]:
### ------ DATA CLEANING  ----------
#cast as string 
other_AB['parent_uprn'] = other_AB['parent_uprn'].astype(str)
other_AB['uprn'] = other_AB['uprn'].astype(str)

#fill in with left side padding zeros 
other_AB['parent_uprn'] = other_AB['parent_uprn'].apply(lambda x: '{0:0>12}'.format(x))
other_AB['uprn'] = other_AB['uprn'].apply(lambda x: '{0:0>12}'.format(x))


In [13]:
### ------ DATA CLEANING On RESI AB ----------

#Address matching data clean, make building number into a string 
resi_AB['building_number'] = resi_AB['building_number'].astype('Int64').astype('str') 

In [14]:
resi_AB['pao_start_number'] = resi_AB['pao_start_number'].astype('Int64').astype('str') 


In [15]:
other_AB['building_number'] = other_AB['building_number'].astype('Int64').astype('str') 

In [16]:
other_AB['pao_start_number'] = other_AB['pao_start_number'].astype('Int64').astype('str') 


------ DATA CLEANING PLANNING APPLICATION DATA  ----------


In [18]:

#BATCH 1 ONLY london_data has extra quotation marks, need to remove
#london_data['uprn'] = london_data['uprn'].apply(lambda x: x.strip("''") if not pd.isna(x) else x)


In [17]:
### ------ DATA CLEANING  ----------
london_data['uprn_x'] = london_data['uprn_x'].astype('Int64')



In [18]:
#cast the strings and make sure it looks good 
london_data['uprn_x'] = london_data['uprn_x'].astype('str') #apply(lambda x: str(x) if not pd.isna(x) else x)

london_data['uprn_x'] = london_data['uprn_x'].apply(lambda x: '{0:0>12}'.format(x) if not pd.isna(x) else x)

In [19]:
### ------ DATA CLEANING  ----------

london_data = london_data.replace('00000000<NA>',np.NaN)

Ignore the creation of the columns below if matching a batch for a second, third, etc. time

In [20]:
### ------ DATA CLEANING  ----------
#street address from site_name_LPA
london_data['parsed_street_LPA'] = london_data['site_name_LPA'].apply(lambda x: re.findall("[0-9]+.-?.[0-9]+?\s(.+)(Road|Lane|Avenue|Parade|Courtyard|Street|Gardens|Drive)", x) if not pd.isna(x) and '-' in x
                                                                  else (re.findall("[0-9]+\s(.+)(Road|Lane|Avenue|Parade|Courtyard|Street|Gardens|Drive)", x) if not pd.isna(x) else []))


In [21]:
london_data['parsed_street_LPA'] =  london_data['parsed_street_LPA'].apply(lambda x: x[0][0] + x[0][1]  if len(x) > 0 else None)

In [22]:
#cleaning - street address
london_data['parsed_street_GLA'] = london_data['site_name_GLA'].apply(lambda x: re.findall("[0-9]+.-?.[0-9]+?\s(.+)(Road|Lane|Avenue|Parade|Courtyard|Street|Gardens|Drive)", x) if not pd.isna(x) and '-' in x
                                                                  else (re.findall("[0-9]+\s(.+)(Road|Lane|Avenue|Parade|Courtyard|Street|Gardens|Drive)", x) if not pd.isna(x) else []))


In [23]:
london_data['parsed_street_GLA'] =  london_data['parsed_street_GLA'].apply(lambda x: x[0][0] + x[0][1]  if len(x) > 0 else None)

In [24]:
#if the original street was empty, then we want to use the LPA Or GLA parsed name as street_name 

london_data['street_name'] = np.where(london_data['street_name'].isnull(), np.where(london_data['parsed_street_LPA'].isnull(), london_data['parsed_street_GLA'], london_data['parsed_street_LPA']), london_data['street_name'] )

In [27]:
# DATA EXPLORATION - EXPORT CSV IF NEEDED 
#london_data.to_csv('batch2_cleaned.csv', index = False)

Continue here to clean data for all batches

In [20]:
# create new column with the number, street description, and the site name 
london_data['concat_addr'] = np.where(london_data['site_name_clean'].isnull(), '', london_data['site_name_clean'] +  ', ' ) + london_data['site_number_clean'] + ', ' +  london_data['street_name']

In [21]:
london_data['concat_addr'] = london_data['concat_addr'].str.upper()

In [22]:
#make london_data all into capitals to regularize
london_data['street_name'] = london_data['street_name'].str.upper()

In [23]:
#remove unwanted characters
london_data['postcode_clean'] = london_data['postcode_clean'].replace('x000D__x000D_\n', '')

In [24]:
london_data = london_data.replace({'_x000D__x000D_\n': ' ', '_x000d__x000d_\n': ' ' }, regex = True)

In [25]:
#remove white space
london_data['postcode_clean'] = london_data['postcode_clean'].apply(lambda x: str(x).strip())

In [27]:
london_data['postcode_sector'] = london_data['postcode_clean'].apply(lambda x: x[:-2])

In [29]:
#substring of site_name_GLA without the postcode .... 
london_data['site_name_GLA_no_pc'] = london_data.apply(lambda row: str(row['site_name_GLA']).upper().replace(', ' + row['postcode_clean'], ''), axis = 1)

In [30]:
london_data['site_name_LPA_no_pc'] = london_data.apply(lambda row: str(row['site_name_LPA']).upper().replace(row['postcode_clean'], ''), axis = 1)


UPRN MATCHING: If reprocessing data that has already been matched, Ignore the Data matching on the UPRN / Parent UPRN

In [32]:
#### -------- DATA MERGING --------
#join the london data with the RESIDENTIAL AddressBase dataset on UPRN
merged = london_data.merge(resi_AB, how = 'left', left_on = 'uprn', right_on = 'parent_uprn')

#merged['UCL_ID'].nunique()

In [33]:
#separate merged into no match and match 
merged_match, merged_no_match = separate_matches(merged, 'parent_uprn', 'parent_uprn')

merged_no_match = merged_no_match.dropna(axis=1, how='all')

In [34]:
#merge 2 on uprn not parent_uprn 
merged_2 = merged_no_match.merge(resi_AB, how = 'left', left_on = 'uprn_x', right_on = 'uprn')

In [35]:
#separate the merge 2 into two datasets for match v not match 
merged_2_match, merged_2_no_match = separate_matches(merged_2, 'uprn', 'uprn')

merged_2_no_match = merged_2_no_match.dropna(axis=1, how='all')      

In [36]:
merged_2_match = merged_2_match.rename(columns={"uprn": "uprn_OSAB"})
merged_match = merged_match.rename(columns={"uprn_y": "uprn_OSAB"})

#merged_2_match.count()

In [37]:
all_matched = pd.concat([merged_2_match, merged_match])


ADDRESS MATCHING: For all matching versions: Address matching starts here

In [31]:
merged_2_no_match = london_data

-------- DATA MERGING --------  STRATEGY 1 ADDRESS MATCH

In [32]:
## ADDRESS STRATEGY ONE 
#use the site name GLA no pc with the parsed address 1 
left_columns = ['site_name_GLA_no_pc', 'postcode_clean']  #['site_number_clean', 'street_name', 'postcode_clean']
right_columns = ['parsed_address1', 'postcode_locator'] #['building_number', 'street_description', 'postcode_locator']

# this is a very strict conservative join 
merged_on_address = merged_2_no_match.merge(resi_AB, how = 'left', left_on = left_columns, right_on = right_columns)

In [33]:
merged_on_address_match,merged_on_address_no_match =  separate_matches(merged_on_address, 'uprn', 'address_1')


In [34]:
#how many unique ids were matched?
merged_on_address_match['ID'].nunique()

68

In [35]:
## Add the merged_on_address_match to the matched
merged_on_address_match = merged_on_address_match.rename(columns={"uprn": "uprn_OSAB"})
merged_on_address_match['match_strategy'] = 'address_1'

all_matched = merged_on_address_match

# Use this if UPRN was used for matching above
#all_matched = pd.concat([all_matched, merged_on_address_match])

merged_on_address_no_match = merged_on_address_no_match.dropna(axis=1, how='all')


In [36]:
#### ADDRESS Strategy 2 - 
## use the site name LPA without post code ...
left_columns_2 = ['site_name_LPA_no_pc', 'postcode_clean'] #London Data 
right_columns_2 =  ['parsed_address1', 'postcode_locator'] #AB

merged_on_address2 = merged_on_address_no_match.merge(resi_AB, how = 'left', left_on = left_columns_2, right_on = right_columns_2)

In [37]:
merged_on_address2_match = merged_on_address2.drop(merged_on_address2[pd.isna(merged_on_address2['uprn']) == True].index) 
merged_on_address2_no_match = merged_on_address2.drop(merged_on_address2[pd.isna(merged_on_address2['uprn']) == False].index) 


In [38]:
merged_on_address2_match['ID'].nunique()

0

In [39]:
merged_on_address2_match = merged_on_address2_match.rename(columns={"uprn": "uprn_OSAB"})
merged_on_address2_match = merged_on_address2_match.drop(columns = ['short_site_name_LPA]']) 
merged_on_address2_match['match_strategy'] = 'address_2'

In [40]:
all_matched = pd.concat([all_matched, merged_on_address2_match])

In [41]:
merged_on_address2_no_match = merged_on_address2_no_match.dropna(axis=1, how='all')


-------- DATA MERGING --------  STRATEGY 3 ADDRESS MATCH

In [42]:
# #-------- DATA MERGING --------  STRATEGY 3 ADDRESS MATCH 
 # use the site name GLA no pc with the parsed address 2 
left_columns_3 = ['site_name_GLA_no_pc', 'postcode_clean'] 
right_columns_3 = ['parsed_address2', 'postcode_locator'] 


merged_on_address3_match, merged_on_address3_no_match = my_merge(merged_on_address2_no_match, resi_AB, left_columns_3, right_columns_3)

In [43]:
#for batch 2, gets 0 results
#merged_on_address3_no_match.count()

In [44]:
# add the matched into all matched
merged_on_address3_match = merged_on_address3_match.rename(columns={"uprn": "uprn_OSAB"})
merged_on_address3_match['match_strategy'] = 'address_3'

frames = [all_matched, merged_on_address3_match]

all_matched = pd.concat(frames)

In [45]:
merged_on_address3_no_match = merged_on_address3_no_match.dropna(axis=1, how='all')

In [46]:
#### ADDRESS Strategy 4 
left_columns_4 = ['site_name_LPA_no_pc', 'postcode_clean']  #London Data 
right_columns_4 = ['parsed_address2', 'postcode_locator'] #AB

merged_on_address4 = merged_on_address3_no_match.merge(resi_AB, how = 'left', left_on = left_columns_4, right_on = right_columns_4)

In [47]:
merged_on_address4_match = merged_on_address4.drop(merged_on_address4[pd.isna(merged_on_address4['uprn']) == True].index) 
merged_on_address4_no_match = merged_on_address4.drop(merged_on_address4[pd.isna(merged_on_address4['uprn']) == False].index) 


In [48]:
merged_on_address4_match['ID'].nunique()

0

In [49]:
merged_on_address4_match = merged_on_address4_match.rename(columns={"uprn": "uprn_OSAB"})
merged_on_address4_match['match_strategy'] = 'address_4'

frames = [all_matched, merged_on_address4_match]

all_matched = pd.concat(frames)
merged_on_address4_no_match = merged_on_address4_no_match.dropna(axis=1, how='all')

In [50]:
## Strategy 5 
#match on the street number, street name, postcode 

left_columns_5 = ['concat_addr', 'lpa_name']
right_columns_5 = ['parsed_address1', 'administrative_area']

merged_on_address5_match, merged_on_address5_no_match = my_merge(merged_on_address4_no_match, resi_AB, left_columns_5, right_columns_5)

In [51]:
merged_on_address5_match['ID'].nunique()

43

In [52]:
merged_on_address5_match = merged_on_address5_match.rename(columns={"uprn": "uprn_OSAB"})
merged_on_address5_match['match_strategy'] = 'address_5'

frames = [all_matched, merged_on_address5_match]

all_matched = pd.concat(frames)
merged_on_address5_no_match = merged_on_address5_no_match.dropna(axis=1, how='all')

In [53]:
## Strategy 6
left_columns_6 = ['concat_addr', 'lpa_name']
right_columns_6 = ['parsed_address2', 'administrative_area']

merged_on_address6_match, merged_on_address6_no_match = my_merge(merged_on_address5_no_match, resi_AB, left_columns_6, right_columns_6)

In [54]:
merged_on_address6_match['ID'].nunique()

32

In [56]:
merged_on_address6_match = merged_on_address6_match.rename(columns={"uprn": "uprn_OSAB"})
merged_on_address6_match['match_strategy'] = 'address_6'

frames = [all_matched, merged_on_address6_match]

all_matched = pd.concat(frames)
merged_on_address6_no_match = merged_on_address6_no_match.dropna(axis=1, how='all')

In [56]:
# pao_start_number + street_description

In [57]:
## Strategy 7
left_columns_7 = ['site_number_clean', 'street_name', 'postcode_clean']
right_columns_7 = ['pao_start_number', 'street_description', 'postcode_locator']

merged_on_address7_match, merged_on_address7_no_match = my_merge(merged_on_address6_no_match, resi_AB, left_columns_7, right_columns_7)

In [58]:
merged_on_address7_match['ID'].nunique()

33

In [60]:
merged_on_address7_match = merged_on_address7_match.rename(columns={"uprn": "uprn_OSAB"})
merged_on_address7_match['match_strategy'] = 'address_7'

frames = [all_matched, merged_on_address7_match]

all_matched = pd.concat(frames)
merged_on_address7_no_match = merged_on_address7_no_match.dropna(axis=1, how='all')

In [65]:
### strategy 8 
left_columns_8 = ['site_number_clean', 'street_name', 'postcode_sector_x']
right_columns_8 = ['pao_start_number', 'street_description', 'postcode_sector']

merged_on_address8_match, merged_on_address8_no_match = my_merge(merged_on_address7_no_match, resi_AB, left_columns_8, right_columns_8)


In [66]:
merged_on_address8_match['ID'].nunique()

71

In [67]:
merged_on_address8_match = merged_on_address8_match.rename(columns={"uprn": "uprn_OSAB"})
merged_on_address8_match['match_strategy'] = 'address_8'

frames = [all_matched, merged_on_address8_match]

all_matched = pd.concat(frames)
merged_on_address8_no_match = merged_on_address8_no_match.dropna(axis=1, how='all')

In [70]:
### strategy 9 
left_columns_9 = ['site_name_GLA_no_pc',  'postcode_sector_x']
right_columns_9 = ['parsed_address1',  'postcode_sector']

merged_on_address9_match, merged_on_address9_no_match = my_merge(merged_on_address8_no_match, resi_AB, left_columns_9, right_columns_9)


In [71]:
merged_on_address9_match['ID'].nunique()

3

In [72]:
merged_on_address9_match = merged_on_address9_match.rename(columns={"uprn": "uprn_OSAB"})
merged_on_address9_match['match_strategy'] = 'address_9'

frames = [all_matched, merged_on_address9_match]

all_matched = pd.concat(frames)
merged_on_address9_no_match = merged_on_address9_no_match.dropna(axis=1, how='all')

In [73]:
### strategy 10 
left_columns_10 = ['site_name_LPA_no_pc',  'postcode_sector_x']
right_columns_10 = ['parsed_address1',  'postcode_sector']

merged_on_address10_match, merged_on_address10_no_match = my_merge(merged_on_address9_no_match, resi_AB, left_columns_10, right_columns_10)


In [74]:
merged_on_address10_match['ID'].nunique()

0

In [75]:
merged_on_address10_match = merged_on_address10_match.rename(columns={"uprn": "uprn_OSAB"})
merged_on_address10_match['match_strategy'] = 'address_10'

frames = [all_matched, merged_on_address10_match]

all_matched = pd.concat(frames)
merged_on_address10_no_match = merged_on_address10_no_match.dropna(axis=1, how='all')

In [76]:
### strategy 11 
left_columns_11 = ['site_name_GLA_no_pc',  'postcode_sector_x']
right_columns_11 = ['parsed_address2',  'postcode_sector']

merged_on_address11_match, merged_on_address11_no_match = my_merge(merged_on_address10_no_match, resi_AB, left_columns_11, right_columns_11)


In [77]:
merged_on_address11_match['ID'].nunique()

0

In [78]:
merged_on_address11_match = merged_on_address11_match.rename(columns={"uprn": "uprn_OSAB"})
merged_on_address11_match['match_strategy'] = 'address_11'

frames = [all_matched, merged_on_address11_match]

all_matched = pd.concat(frames)
merged_on_address11_no_match = merged_on_address11_no_match.dropna(axis=1, how='all')

In [79]:
### strategy 12 
left_columns_12 = ['site_name_LPA_no_pc',  'postcode_sector_x']
right_columns_12 = ['parsed_address2',  'postcode_sector']

merged_on_address12_match, merged_on_address12_no_match = my_merge(merged_on_address11_no_match, resi_AB, left_columns_12, right_columns_12)


In [80]:
merged_on_address12_match['ID'].nunique()

0

In [81]:
### strategy 13 
left_columns_13 = ['site_number_clean', 'street_name',  'postcode_sector_x']
right_columns_13 = ['pao_start_number', 'street_description',   'postcode_sector']


merged_on_address13_match, merged_on_address13_no_match = my_merge(merged_on_address11_no_match, resi_AB, left_columns_13, right_columns_13)


In [82]:
merged_on_address13_match['ID'].nunique()

0

------ specific address analysis ----- 

In [66]:
merged_on_address7_match.head(20)

,ID,planning_application_number,lpa_name,application_type,application_type_full,description,number_of_units,site_number_clean,street_name,postcode_clean,...,dependent_locality,locality,town_name,administrative_area,post_town,postcode,postcode_locator,parsed_address1,parsed_address2,match_strategy
7,1547,P2017/2905/PRA,ISLINGTON,Prior Approval,Prior Approval (Class M - formerly IA),Notification for Prior Approval for the change...,1.0,194,SEVEN SISTERS ROAD,N4 3NX,...,0,0,LONDON,ISLINGTON,LONDON,N4 3NX,N4 3NX,"FLAT 3, 194, SEVEN SISTERS ROAD","FLAT 3, 194A, SEVEN SISTERS ROAD",address_6
8,1547,P2017/2905/PRA,ISLINGTON,Prior Approval,Prior Approval (Class M - formerly IA),Notification for Prior Approval for the change...,1.0,194,SEVEN SISTERS ROAD,N4 3NX,...,0,0,LONDON,ISLINGTON,LONDON,N4 3NX,N4 3NX,"FLAT 1, 194, SEVEN SISTERS ROAD","FLAT 1, 194A, SEVEN SISTERS ROAD",address_6
9,1547,P2017/2905/PRA,ISLINGTON,Prior Approval,Prior Approval (Class M - formerly IA),Notification for Prior Approval for the change...,1.0,194,SEVEN SISTERS ROAD,N4 3NX,...,0,0,LONDON,ISLINGTON,LONDON,N4 3NX,N4 3NX,"FLAT 2, 194, SEVEN SISTERS ROAD","FLAT 2, 194A, SEVEN SISTERS ROAD",address_6
10,1547,P2016/2604/PRA,ISLINGTON,Prior Approval,Prior Approval (Class O - formerly J),Prior approval application for the proposed ch...,1.0,222,SEVEN SISTERS ROAD,N4 3NX,...,0,0,LONDON,ISLINGTON,0,0,N4 3NX,"FLAT 5, 222, SEVEN SISTERS ROAD",SEVEN SISTERS ROAD,address_6
11,1547,P2016/2604/PRA,ISLINGTON,Prior Approval,Prior Approval (Class O - formerly J),Prior approval application for the proposed ch...,1.0,222,SEVEN SISTERS ROAD,N4 3NX,...,0,0,LONDON,ISLINGTON,0,0,N4 3NX,"FLAT 3, 222, SEVEN SISTERS ROAD",SEVEN SISTERS ROAD,address_6
12,1547,P2016/2604/PRA,ISLINGTON,Prior Approval,Prior Approval (Class O - formerly J),Prior approval application for the proposed ch...,1.0,222,SEVEN SISTERS ROAD,N4 3NX,...,0,0,LONDON,ISLINGTON,0,0,N4 3NX,"FLAT 1, 222, SEVEN SISTERS ROAD",SEVEN SISTERS ROAD,address_6
13,1547,P2016/2604/PRA,ISLINGTON,Prior Approval,Prior Approval (Class O - formerly J),Prior approval application for the proposed ch...,1.0,222,SEVEN SISTERS ROAD,N4 3NX,...,0,0,LONDON,ISLINGTON,0,0,N4 3NX,"FLAT 2, 222, SEVEN SISTERS ROAD",SEVEN SISTERS ROAD,address_6
14,1547,P2016/2604/PRA,ISLINGTON,Prior Approval,Prior Approval (Class O - formerly J),Prior approval application for the proposed ch...,1.0,222,SEVEN SISTERS ROAD,N4 3NX,...,0,0,LONDON,ISLINGTON,0,0,N4 3NX,"FLAT 4, 222, SEVEN SISTERS ROAD",SEVEN SISTERS ROAD,address_6
15,1547,P2016/2604/PRA,ISLINGTON,Prior Approval,Prior Approval (Class O - formerly J),Prior approval application for the proposed ch...,1.0,222,SEVEN SISTERS ROAD,N4 3NX,...,0,0,LONDON,ISLINGTON,0,0,N4 3NX,"FLAT 5, 222, SEVEN SISTERS ROAD",SEVEN SISTERS ROAD,address_6
16,1547,P2016/2604/PRA,ISLINGTON,Prior Approval,Prior Approval (Class O - formerly J),Prior approval application for the proposed ch...,1.0,222,SEVEN SISTERS ROAD,N4 3NX,...,0,0,LONDON,ISLINGTON,0,0,N4 3NX,"FLAT 3, 222, SEVEN SISTERS ROAD",SEVEN SISTERS ROAD,address_6


In [62]:
other_AB[(other_AB['building_number'] == '3') & (other_AB['street_description'] == 'HIGH STREET') & (other_AB['postcode_locator'] == 'BR1 1LF')]

,uprn,class,parent_uprn,latitude,longitude,country,legal_name,sub_building_name,building_name,building_number,...,street_description,dependent_locality,locality,town_name,administrative_area,post_town,postcode,postcode_locator,parsed_address1,parsed_address2


In [178]:
### hmm why isnt itfinding the correct record?? 
#this record exists in other AB - okay... so might work once I match taht! 
other_AB[(other_AB['street_description'] == 'SEVEN SISTERS ROAD')  & (other_AB['postcode_locator'] == 'N4 3NX') 
                        & (other_AB['parsed_address1'] == '194, SEVEN SISTERS ROAD')]

,uprn,class,parent_uprn,latitude,longitude,country,legal_name,sub_building_name,building_name,building_number,...,street_description,dependent_locality,locality,town_name,administrative_area,post_town,postcode,postcode_locator,parsed_address1,parsed_address2
4055311,005300082025,CR,000000000000,51.562168,-0.109969,E,0.0,0,0,194,...,SEVEN SISTERS ROAD,0,0,LONDON,ISLINGTON,LONDON,N4 3NX,N4 3NX,"194, SEVEN SISTERS ROAD","194, SEVEN SISTERS ROAD"
4136519,005300082026,PP,000000000000,51.562168,-0.109969,E,0.0,0,0,<NA>,...,SEVEN SISTERS ROAD,0,0,LONDON,ISLINGTON,0,0,N4 3NX,"194, SEVEN SISTERS ROAD",SEVEN SISTERS ROAD


In [171]:
seven_sisters.to_csv('seven_sisters.csv', index=False) 

In [60]:
##-------- DATA MERGING --------  COMMERCIAL PROPERTIES --- Reproduce the process with commercial properties 
#Merge on Parent UPRN 
non_resi_merged = merged_on_address3_no_match.merge(other_AB,how = 'left', left_on = 'uprn_x', right_on = 'parent_uprn')

non_resi_match, non_resi_no_match =  separate_matches(non_resi_merged, 'parent_uprn', 'parent_uprn')

non_resi_no_match = non_resi_no_match.dropna(axis=1, how='all')

In [61]:
#merge on UPRN
non_resi_merged_2 = non_resi_no_match.merge(other_AB, how = 'left', left_on = 'uprn_x', right_on = 'uprn')

non_resi_match_2, non_resi_no_match_2 =  separate_matches(non_resi_merged_2, 'parent_uprn', 'uprn')

non_resi_no_match_2 = non_resi_no_match_2.dropna(axis=1, how='all')

In [62]:
non_resi_match_2 = non_resi_match_2.rename(columns={"uprn": "uprn_OSAB"})
#non_resi_match_2['match_strategy'] = 'uprn'
non_resi_match_2.count()

non_resi_match = non_resi_match.rename(columns={"uprn": "uprn_OSAB"})
#non_resi_match['match_strategy'] = 'parent_uprn'

In [63]:
#union them together 
non_resi_all_matched = pd.concat([non_resi_match_2, non_resi_match])

---- Commercial address matching --- 

In [83]:
#now join on addresses strat 1 
left_columns = ['site_name_GLA_no_pc', 'postcode_clean']  #['site_number_clean', 'street_name', 'postcode_clean']
right_columns = ['parsed_address1', 'postcode_locator'] #['building_number', 'street_description', 'postcode_locator']

#non_resi_address_merge_match, non_resi_address_merge_no_match = my_merge(non_resi_no_match_2, other_AB, left_columns, right_columns)

In [84]:
#use this one if skipping UPRN 
non_resi_address_merge_match, non_resi_address_merge_no_match = my_merge(merged_on_address7_no_match, other_AB, left_columns, right_columns)


In [86]:
non_resi_address_merge_no_match = non_resi_address_merge_no_match.dropna(axis=1, how='all')

## Add the merged_on_address_match to the matched
non_resi_address_merge_match = non_resi_address_merge_match.rename(columns={"uprn": "uprn_OSAB"})
non_resi_address_merge_match['match_strategy'] = 'address_1'

non_resi_all_matched = non_resi_address_merge_match

In [85]:
non_resi_address_merge_match['ID'].nunique()

73

In [96]:
#USE THIS FIRST TIME THRU 
# non_resi_all_matched = pd.concat([non_resi_all_matched, non_resi_address_merge_match])

In [87]:
#join addresses strat 2
left_columns_2 =  ['site_name_LPA_no_pc', 'postcode_clean'] 
right_columns_2 =  ['parsed_address1', 'postcode_locator']
 
non_resi_address_merge_match2, non_resi_address_merge_no_match2 = my_merge(non_resi_address_merge_no_match, other_AB, left_columns_2,right_columns_2 )

In [88]:
non_resi_address_merge_match2['ID'].nunique()

0

In [89]:
non_resi_address_merge_no_match2 = non_resi_address_merge_no_match2.dropna(axis=1, how='all')

## Add the merged_on_address_match to the matched
non_resi_address_merge_match2 = non_resi_address_merge_match2.rename(columns={"uprn": "uprn_OSAB"})
non_resi_address_merge_match2['match_strategy'] = 'address_2'

non_resi_all_matched = pd.concat([non_resi_all_matched, non_resi_address_merge_match2])

In [90]:
# STRATEGY 3 ADDRESS MATCH 
left_columns_3 = ['site_name_GLA_no_pc', 'postcode_clean'] 
right_columns_3 = ['parsed_address2', 'postcode_locator']


non_resi_address_merge_match3, non_resi_address_merge_no_match3 = my_merge(non_resi_address_merge_no_match2, other_AB, left_columns_3, right_columns_3)

In [91]:
non_resi_address_merge_match3['ID'].nunique()

93

In [92]:
non_resi_address_merge_no_match3 = non_resi_address_merge_no_match3.dropna(axis=1, how='all')

## Add the merged_on_address_match to the matched
non_resi_address_merge_match3 = non_resi_address_merge_match3.rename(columns={"uprn": "uprn_OSAB"})
non_resi_address_merge_match3['match_strategy'] = 'address_3'

non_resi_all_matched = pd.concat([non_resi_all_matched, non_resi_address_merge_match3])

------- ADDRESS ----- Strategy 4 COMM 

In [93]:
# STRATEGY 4 ADDRESS MATCH 
left_columns_4 = ['site_name_LPA_no_pc', 'postcode_clean'] 
right_columns_4 = ['parsed_address2', 'postcode_locator']


non_resi_address_merge_match4, non_resi_address_merge_no_match4 = my_merge(non_resi_address_merge_no_match3, other_AB, left_columns_4, right_columns_4)

In [94]:
non_resi_address_merge_match4['ID'].nunique()

0

In [95]:
non_resi_address_merge_no_match4 = non_resi_address_merge_no_match4.dropna(axis=1, how='all')

## Add the merged_on_address_match to the matched
non_resi_address_merge_match4 = non_resi_address_merge_match4.rename(columns={"uprn": "uprn_OSAB"})
non_resi_address_merge_match4['match_strategy'] = 'address_4'

non_resi_all_matched = pd.concat([non_resi_all_matched, non_resi_address_merge_match4])

In [96]:
#strategy 5 ADDRESS MATCH 
left_columns_5 = ['concat_addr', 'lpa_name'] 
right_columns_5 = ['parsed_address1', 'administrative_area']


non_resi_address_merge_match5, non_resi_address_merge_no_match5 = my_merge(non_resi_address_merge_no_match4, other_AB, left_columns_5, right_columns_5)


In [97]:
non_resi_address_merge_no_match5 = non_resi_address_merge_no_match5.dropna(axis=1, how='all')

## Add the merged_on_address_match to the matched
non_resi_address_merge_match5 = non_resi_address_merge_match5.rename(columns={"uprn": "uprn_OSAB"})
non_resi_address_merge_match5['match_strategy'] = 'address_5'

non_resi_all_matched = pd.concat([non_resi_all_matched, non_resi_address_merge_match5])

strategy 6, same as for resi --- 

In [98]:
## Strategy 6
left_columns_6 = ['concat_addr', 'lpa_name']  
right_columns_6 = ['parsed_address2', 'administrative_area'] 

non_resi_address_merge_match6, non_resi_address_merge_no_match6 = my_merge(non_resi_address_merge_no_match5, other_AB, left_columns_6, right_columns_6)

In [99]:
non_resi_address_merge_match6['ID'].nunique()

32

In [100]:
non_resi_address_merge_no_match6 = non_resi_address_merge_no_match6.dropna(axis=1, how='all')

## Add the merged_on_address_match to the matched
non_resi_address_merge_match6 = non_resi_address_merge_match6.rename(columns={"uprn": "uprn_OSAB"})
non_resi_address_merge_match6['match_strategy'] = 'address_6'

non_resi_all_matched = pd.concat([non_resi_all_matched, non_resi_address_merge_match6])

In [101]:
#strategy 7 ADDRESS MATCH 
left_columns_7 = ['site_number_clean', 'street_name', 'postcode_clean']
right_columns_7 = ['pao_start_number', 'street_description', 'postcode_locator']

non_resi_address_merge_match7, non_resi_address_merge_no_match7 = my_merge(non_resi_address_merge_no_match6, other_AB, left_columns_7, right_columns_7)


In [102]:
non_resi_address_merge_match7['ID'].nunique() #was 88

9

In [103]:
non_resi_address_merge_no_match7 = non_resi_address_merge_no_match7.dropna(axis=1, how='all')

## Add the merged_on_address_match to the matched
non_resi_address_merge_match7 = non_resi_address_merge_match7.rename(columns={"uprn": "uprn_OSAB"})
non_resi_address_merge_match7['match_strategy'] = 'address_7'

non_resi_all_matched = pd.concat([non_resi_all_matched, non_resi_address_merge_match7])

In [105]:
#strategy 8 ADDRESS MATCH 
left_columns_8 = ['site_number_clean', 'street_name', 'postcode_sector_x']
right_columns_8 = ['pao_start_number', 'street_description', 'postcode_sector']

non_resi_address_merge_match8, non_resi_address_merge_no_match8 = my_merge(non_resi_address_merge_no_match7, other_AB, left_columns_8, right_columns_8)

In [106]:
non_resi_address_merge_match8['ID'].nunique() #was 88

33

In [107]:
non_resi_address_merge_no_match8 = non_resi_address_merge_no_match8.dropna(axis=1, how='all')

## Add the merged_on_address_match to the matched
non_resi_address_merge_match8 = non_resi_address_merge_match8.rename(columns={"uprn": "uprn_OSAB"})
non_resi_address_merge_match8['match_strategy'] = 'address_8'

non_resi_all_matched = pd.concat([non_resi_all_matched, non_resi_address_merge_match8])

In [108]:
#strategy 9 ADDRESS MATCH 
left_columns_9 = ['site_name_GLA_no_pc',  'postcode_sector_x']
right_columns_9 = ['parsed_address1',  'postcode_sector']

non_resi_address_merge_match9, non_resi_address_merge_no_match9 = my_merge(non_resi_address_merge_no_match8, other_AB, left_columns_9, right_columns_9)

In [109]:
non_resi_address_merge_match9['ID'].nunique() #was 88

15

In [110]:
non_resi_address_merge_no_match9 = non_resi_address_merge_no_match9.dropna(axis=1, how='all')

## Add the merged_on_address_match to the matched
non_resi_address_merge_match9 = non_resi_address_merge_match9.rename(columns={"uprn": "uprn_OSAB"})
non_resi_address_merge_match9['match_strategy'] = 'address_9'

non_resi_all_matched = pd.concat([non_resi_all_matched, non_resi_address_merge_match9])

In [111]:
#STRATEGY 10
left_columns_10 = ['site_name_LPA_no_pc',  'postcode_sector_x']
right_columns_10 = ['parsed_address1',  'postcode_sector']

non_resi_address_merge_match10, non_resi_address_merge_no_match10 = my_merge(non_resi_address_merge_no_match9, other_AB, left_columns_10, right_columns_10)

In [112]:
non_resi_address_merge_match10['ID'].nunique() #was 88

0

In [114]:
#STRATEGY 11
left_columns_11 = ['site_name_GLA_no_pc',  'postcode_sector_x']
right_columns_11 = ['parsed_address2',  'postcode_sector']

non_resi_address_merge_match11, non_resi_address_merge_no_match11 = my_merge(non_resi_address_merge_no_match9, other_AB, left_columns_11, right_columns_11)

In [115]:
non_resi_address_merge_match11['ID'].nunique() #was 88

6

In [116]:
non_resi_address_merge_no_match11 = non_resi_address_merge_no_match11.dropna(axis=1, how='all')

## Add the merged_on_address_match to the matched
non_resi_address_merge_match11 = non_resi_address_merge_match11.rename(columns={"uprn": "uprn_OSAB"})
non_resi_address_merge_match11['match_strategy'] = 'address_11'

non_resi_all_matched = pd.concat([non_resi_all_matched, non_resi_address_merge_match11])

In [117]:
#STRATEGY 12
left_columns_12 = ['site_name_LPA_no_pc',  'postcode_sector_x']
right_columns_12 = ['parsed_address2',  'postcode_sector']

non_resi_address_merge_match12, non_resi_address_merge_no_match12 = my_merge(non_resi_address_merge_no_match11, other_AB, left_columns_12, right_columns_12)

In [118]:
non_resi_address_merge_match12['ID'].nunique() #was 88

0

In [119]:
#STRATEGY 13
left_columns_13 = ['site_number_clean', 'street_name',  'postcode_sector_x']
right_columns_13 = ['pao_start_number', 'street_description',   'postcode_sector']

non_resi_address_merge_match13, non_resi_address_merge_no_match13 = my_merge(non_resi_address_merge_no_match11, other_AB, left_columns_13, right_columns_13)

In [120]:
non_resi_address_merge_match13['ID'].nunique() #was 88

0

In [121]:
non_resi_address_merge_no_match13 = non_resi_address_merge_no_match13.dropna(axis=1, how='all')

## Add the merged_on_address_match to the matched
non_resi_address_merge_match13 = non_resi_address_merge_match13.rename(columns={"uprn": "uprn_OSAB"})
non_resi_address_merge_match13['match_strategy'] = 'address_13'

non_resi_all_matched = pd.concat([non_resi_all_matched, non_resi_address_merge_match13])

----- specific address analysis  ---- 

In [71]:
## ------------------------------- Post Match  -------------------------------

In [122]:
#print match rate
resi_match_rate = 100 * all_matched['ID'].nunique() / london_data['ID'].nunique()

print('Resi Match rate: ', resi_match_rate) 

Resi Match rate:  16.72374429223744


In [124]:
all_matched['ID'].nunique()

293

In [123]:
other_match_rate = 100 * non_resi_all_matched['ID'].nunique() / london_data['ID'].nunique()

print('Other Match rate: ', other_match_rate) 

Other Match rate:  17.69406392694064


In [125]:
non_resi_all_matched['ID'].nunique()

310

In [126]:
#before export, make all these columns blank instead of 0 
#'legal_name', 'sub_building_name', 'building_name','building_number','street_description', 'dependent_locality	locality
all_matched['street_name'] = all_matched['street_name'].replace(0,'')
all_matched['legal_name'] = all_matched['legal_name'].replace(0,'')
all_matched['sub_building_name'] = all_matched['sub_building_name'].replace(0,'')
all_matched['building_name'] = all_matched['building_name'].replace(0,'')
all_matched['building_number'] = all_matched['building_number'].replace(0,'')
all_matched['street_description'] = all_matched['street_description'].replace(0,'')
all_matched['dependent_locality'] = all_matched['dependent_locality'].replace(0,'')
all_matched['locality'] = all_matched['locality'].replace(0,'')
all_matched['post_town'] = all_matched['post_town'].replace(0,'')
all_matched['postcode'] = np.where(all_matched['postcode'] == 0, all_matched['postcode_clean'], all_matched['postcode'])


In [127]:
#write out to csv 

all_matched.to_csv('London2b_address_matched_1705.csv', index=False) 

non_resi_all_matched.to_csv('London2b_non_resi_match_1705.csv', index=False)  

In [128]:
non_resi_address_merge_no_match6.to_csv('London2b_no_match_1705.csv', index = False)

In [129]:
#### some analysis on past records
#London2_no_match_2603.csv
#London2_non_resi_match_2603.csv
#London2_address_matched_2603.csv

batch2_no_match = pd.read_csv('data/London2_no_match_2603.csv')

In [130]:
batch2_non_resi = pd.read_csv('data/London2_non_resi_match_2603.csv')
batch2_resi = pd.read_csv('data/London2_address_matched_2603.csv')

In [133]:
batch2_resi.nunique() #1752 no match , 676 non resi , 729 resi

ID                              729
planning_application_number     990
lpa_name                         37
application_type                  1
application_type_full            17
description                     943
number_of_units                  59
site_number_clean               330
street_name                     537
postcode_clean                  695
site_name_clean                 156
site_name_GLA                   747
site_name_LPA                   839
uprn_x                          228
decision                         57
status                           10
application_date                585
decision_date                   667
parsed_street_LPA               345
parsed_street_GLA                17
uprn_OSAB                      5560
class                            10
parent_uprn                     477
latitude                        980
longitude                       979
country                           1
legal_name                        0
sub_building_name           